In [ ]:
from nidaqmx import Task
from nidaqmx.constants import Edge, AcquisitionType
from nidaqmx.stream_writers import AnalogMultiChannelWriter
import numpy as np

import tifffile as tiff
from pathlib import Path
from scipy import fft

import time

import matplotlib.pyplot as plt

In [ ]:
# Creating a task controlling the z galvo, piezo and camera 
cal_task = Task()
cal_task.ao_channels.add_ao_voltage_chan("Dev1/ao0:2", min_val=-10, max_val=10)

# Generating a lightsheet 
cal_task_xy = Task()
cal_task_xy.ao_channels.add_ao_voltage_chan("Dev2/ao2", min_val=-10, max_val=10)
cal_task_xy.timing.cfg_samp_clk_timing(
    rate=40000,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    #sample_mode=AcquisitionType.FINITE,
    samps_per_chan=10000,
)

In [ ]:
xy_val = 2
xy = np.linspace(-xy_val,xy_val,200)
xy = np.append(xy, np.linspace(xy_val,-xy_val,200))

cal_task_xy.write(xy, auto_start=True)

In [ ]:
piezo_val = np.linspace(1, 4, 5)
possible_z_val = np.linspace(-1, 1,30)#30)
                                            
start_time = time.time_ns()
for piezopos in piezo_val:
    for zgalvopos in possible_z_val:    
        cam_pulse = 5

        # run and get camera image
        cal_array = np.stack([0, zgalvopos, piezopos, cam_pulse])
        cal_task.write(cal_array, auto_start=True)
        
        time.sleep(0.1)
        cam_pulse = 0
        cal_array = np.stack([0, zgalvopos, piezopos, cam_pulse])
        cal_task.write(cal_array, auto_start=True)

print((time.time_ns() - start_time)*1e-9)

In [ ]:
cal_task_xy.stop()

In [ ]:
master = Path(r"C:\Users\Admin\Documents")
path = master / "calibration_image.tif" # name should be set in the camera software 
stack = tiff.imread(path)
print(np.shape(stack))
nx, ny = np.shape(stack)[1:]

In [ ]:
z_val = np.zeros((np.shape(piezo_val)[0], np.shape(possible_z_val)[0]))
optimal_z = np.zeros(np.shape(piezo_val))
print(np.shape(z_val))
count = 0

for piezopos in range(np.shape(piezo_val)[0]):
    for zgalvopos in range(np.shape(possible_z_val)[0]):
        # load tiffs and get fft 
        img = stack[count]
        img_fft = abs(fft.fftshift(fft.fft2(img)))
        
        # choose the best fft
        z_val[piezopos, zgalvopos]  = np.mean(img_fft[0:200] + np.mean(img_fft[:, 0:200]))
        count += 1
    optimal_z[piezopos] = np.argmax(z_val[piezopos])

In [ ]:
# Optimal z galvo position for each piezo position 
optimal_z